In [ ]:
from pathlib import Path

import matplotlib as mpl

mpl.use("pdf")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gsd_metrics import SCALES, GSD_FACTOR
from gsd_utils import papermode

papermode(plt=plt, size=10)

# width as measured in inkscape
mm = 1/25.4
width0 = 138.6 * mm
width = width0
height = width / 1.618

In [ ]:
patch_size = 184
subset = "test"

In [ ]:
# Paths
results_dir = Path.cwd() / "results"
gsd_resdir = results_dir / "gsd" / subset / "resize"
df = pd.read_csv(gsd_resdir / f"multigsd-{subset}-p{patch_size}.csv")

figs_path = Path("figs") / "gsds"
figs_path.mkdir(exist_ok=True, parents=True)

In [ ]:
df

In [ ]:
if False:
    fig, axs = plt.subplots(nrows=3, sharex=True)
    fig.subplots_adjust(left=0.17, bottom=0.17, right=0.99, top=0.97)

    ious = [100 * df[df.GSD == gsd].iou for gsd in df.GSD.unique()]
    accs = [100 * df[df.GSD == gsd].acc for gsd in df.GSD.unique()]
    f1s = [100 * df[df.GSD == gsd].f1 for gsd in df.GSD.unique()]

    m_IOU = [iou.mean() for iou in ious]
    m_acc = [acc.mean() for acc in accs]
    m_f1 = [f1.mean() for f1 in f1s]

    vio_ious = axs[0].violinplot(ious, showmeans=False)
    vio_accs = axs[1].violinplot(accs, showmeans=False)
    vio_f1s = axs[2].violinplot(f1s, showmeans=False)
    inv_scales = 1 / SCALES
    xlabs = ["$1.05$ mm\n(${GSD}_{base}$)"]
    gsdsnum = inv_scales[:, None] @ np.array([1, 1.05])[None, :]
    for gsds in gsdsnum.round(2)[1:, :]:
        gsd, avg = gsds
        xlabs.append(f"${avg}$ mm\n(${gsd}\\times {{GSD}}_{{base}}$)")

    for pc_iou in vio_ious["bodies"]:
        pc_iou.set_facecolor("xkcd:grass green")
        pc_iou.set_edgecolor("xkcd:grass green")
        pc_iou.set_alpha(0.75)

    for pc_acc in vio_accs["bodies"]:
        pc_acc.set_facecolor("xkcd:darkish blue")
        pc_acc.set_edgecolor("xkcd:darkish blue")
        pc_acc.set_alpha(0.75)

    for pc_f1 in vio_f1s["bodies"]:
        pc_f1.set_facecolor("xkcd:orange")
        pc_f1.set_edgecolor("xkcd:orange")
        pc_f1.set_alpha(0.75)

    xticks = 1 + np.arange(SCALES.size)

    axs[0].plot(xticks, m_IOU, "k")
    axs[1].plot(xticks, m_acc, "k")
    axs[2].plot(xticks, m_f1, "k")

    axs[2].set_xticks(xticks, xlabs)
    axs[2].set_xlabel("Ground Sample Distance (m)")

    axs[0].set_ylabel("IoU (\%)")
    axs[1].set_ylabel("Pixel Accuracy (\%)")
    axs[2].set_ylabel("F1 score (\%)")

    axs[0].set_title(f"Patch size: {patch_size}")

    axs[0].set_ylim(bottom=0, top=100)
    axs[1].set_ylim(bottom=0, top=100)
    axs[2].set_ylim(bottom=0, top=100)

    fig.set_size_inches(width, height)

In [ ]:
# fig, axs = plt.subplots(nrows=2, sharex=True)
fig, ax = plt.subplots(squeeze=False)
axs = ax[0]

inv_scales = 1 / SCALES
xlabs = ["$1.05$\n${GSD}_{base}$"]
gsdsnum = inv_scales[:, None] @ np.array([1, 1.05])[None, :]
for gsds in gsdsnum.round(2)[1:, :]:
    gsd, avg = gsds
    xlabs.append(f"${avg}$\n${gsd}\\times {{GSD}}_{{base}}$")

win_ids = df.WIN.unique()[:2]
cmap = plt.get_cmap("viridis", len(win_ids) + 1)
for win_id, col in zip(win_ids, cmap.colors):
    win_df = df[df.WIN == win_id].sort_values("GSD")
    win_size = win_df.winsize.unique().item()
    win_idx = int(win_id[-1])
    win_lab = f"{2 ** win_idx}/{3 ** win_idx}" if win_idx > 0 else 1
    win_lab = f"{win_size} px ({win_lab})"
    win_lab = f"{win_size} px"
    axs[0].plot(
        gsdsnum[:, 1],
        100 * win_df.f1,
        color=col,
        label=win_lab,
        marker="o",
    )
    # axs[1].plot(
    #     gsdsnum[:, 1],
    #     100 * win_df.acc,
    #     color=col,
    #     label=win_lab,
    #     marker="o",
    # )


axs[0].set_ylabel("$F1$ score on \\hspace{2.2em} (\%)")
axs[0].yaxis.set_label_coords(-0.08, 0.5)
axs[0].text(-0.0795, 0.645, r"$D_{test}^{drone}$", fontsize=8, color="blue", transform=axs[0].transAxes, ha='right',
        va='center', rotation=90)
# axs[1].set_ylabel(
#     "Average pixel\naccuracy (\%)",
# )
# axs[1].yaxis.set_label_coords(-0.1, 0.5)

axs[0].set_xlabel("Ground Sample Distance (mm)")
axs[0].xaxis.set_label_coords(0.5, -0.28)


# axs[0].tick_params(bottom=False)

axs[0].set_ylim(bottom=0, top=45)
# axs[1].set_ylim(bottom=0, top=60)

axs[0].set_xscale("log", base=GSD_FACTOR)
axs[0].xaxis.set_major_formatter(plt.ScalarFormatter())
axs[0].set_xticks(gsdsnum[:, 1], xlabs)

axs[0].yaxis.set_major_locator(plt.MultipleLocator(20))
axs[0].yaxis.set_minor_locator(plt.MultipleLocator(10))
# axs[1].yaxis.set_major_locator(plt.MultipleLocator(20))
# axs[1].yaxis.set_minor_locator(plt.MultipleLocator(10))

axs[0].grid(visible=True, which="major", axis="both")
# axs[1].grid(visible=True, which="major", axis="both")
axs[0].grid(visible=True, which="minor", axis="y", linestyle="--")
# axs[1].grid(visible=True, which="minor", axis="y", linestyle="--")

h, l = axs[0].get_legend_handles_labels()
axs[0].legend(
    h,
    l,
    ncols=len(l),
    loc="lower right",
    # loc="lower center",
    # bbox_to_anchor=(0.5, 1),
    # borderaxespad=0.1,
)

fig.subplots_adjust(left=0.12, bottom=0.30, right=0.95, top=0.99, hspace=0.2)
# fig.legend(h, l, loc = (0.95, 0.5))

fig.set_size_inches(width, 0.75 * height)
# fig.set_size_inches(width, 1.25 * height)
fig.savefig(figs_path / f"multigsd-{subset}-{patch_size}.png")
fig.savefig(figs_path / f"multigsd-{subset}-{patch_size}.pdf")